In [52]:
import datetime as dt
from marshmallow import Schema, fields, post_load
from pprint import pprint


class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        self.created_at = dt.datetime.now()

    def __repr__(self):
        return "<User(name={self.name!r})>".format(self=self)


class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email()
    created_at = fields.DateTime()


## Dumping - serializing

In [48]:
user = User(name="Monty", email="monty@python.org")
schema = UserSchema()
result = schema.dump(user)
pprint(result)

{'created_at': '2020-09-15T09:36:58.525745',
 'email': 'monty@python.org',
 'name': 'Monty'}


In [49]:
json_result = schema.dumps(user)
pprint(json_result)

('{"created_at": "2020-09-15T09:36:58.525745", "name": "Monty", "email": '
 '"monty@python.org"}')


### Filtering output

In [50]:
summary_schema = UserSchema(only=("name", "email"))
pprint(summary_schema.dump(user))

{'email': 'monty@python.org', 'name': 'Monty'}


## Loading - deserializing

In [53]:
user_data = {
    "created_at": "2014-08-11T05:26:03.869245",
    "email": "ken@yahoo.com",
    "name": "Ken",
}
schema = UserSchema()
result = schema.load(user_data)
pprint(result)

{'created_at': datetime.datetime(2014, 8, 11, 5, 26, 3, 869245),
 'email': 'ken@yahoo.com',
 'name': 'Ken'}


É possivel deserializar o dict para um objeto, com o post_load

In [54]:
class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email()
    created_at = fields.DateTime(dump_only=True)

    @post_load
    def make_user(self, data, **kwargs):
        print(data)
        return User(**data)

In [55]:
user_data = {"name": "Ronnie", "email": "ronnie@stones.com"}
schema = UserSchema()
result = schema.load(user_data)

{'name': 'Ronnie', 'email': 'ronnie@stones.com'}


In [56]:
print(result)  # => <User(name='Ronnie')>

<User(name='Ronnie')>


In [57]:
User(**user_data)

<User(name='Ronnie')>

List of objects - Serialization and deserialization

In [58]:
user1 = User(name="Mick", email="mick@stones.com")
user2 = User(name="Keith", email="keith@stones.com")
users = [user1, user2]

In [59]:
schema = UserSchema(many=True)
result = schema.dump(users)  # OR UserSchema().dump(users, many=True)
pprint(result)

[{'created_at': '2020-09-15T09:38:10.046919',
  'email': 'mick@stones.com',
  'name': 'Mick'},
 {'created_at': '2020-09-15T09:38:10.047142',
  'email': 'keith@stones.com',
  'name': 'Keith'}]


or

In [60]:
result = UserSchema().dump(users, many=True)
pprint(result)

[{'created_at': '2020-09-15T09:38:10.046919',
  'email': 'mick@stones.com',
  'name': 'Mick'},
 {'created_at': '2020-09-15T09:38:10.047142',
  'email': 'keith@stones.com',
  'name': 'Keith'}]


In [65]:
for _ in result:
    del _['created_at']

In [66]:
result

[{'name': 'Mick', 'email': 'mick@stones.com'},
 {'name': 'Keith', 'email': 'keith@stones.com'}]

In [67]:
schema = UserSchema()
result_object = schema.load(result, many=True)

{'name': 'Mick', 'email': 'mick@stones.com'}
{'name': 'Keith', 'email': 'keith@stones.com'}


## Specifying Serialization/Deserialization Keys


In [69]:
class UserSchema(Schema):
    name = fields.String()
    email = fields.Email(data_key="emailAddress")


s = UserSchema()

In [72]:
data = {"name": "Mike", "email": "foo@bar.com"}
result = s.dump(data)
print(result)

{'name': 'Mike', 'emailAddress': 'foo@bar.com'}


In [73]:
data = {"name": "Mike", "emailAddress": "foo@bar.com"}
result = s.load(data)
print(result)

{'name': 'Mike', 'email': 'foo@bar.com'}
